In [ ]:
import pandas as pd
import sqlite3
import os 

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

os.chdir('/Users/louise/Documents/PJTS/SQL/Formation/SQL_Explorer/Golden Age of Video Games')
conn= sqlite3.connect("datacamp_python.db")

In [2]:
df_games = pd.read_csv("game_sales_data.csv", encoding="ISO-8859-1")
df_columns = df_games.columns
df_games.head(5)

,Rank,Name,Platform,Publisher,Developer,Critic_Score,User_Score,Total_Shipped,Year
0,1,Wii Sports,Wii,Nintendo,Nintendo EAD,7.7,8.0,82.90,2006
1,2,Super Mario Bros.,NES,Nintendo,Nintendo EAD,10.0,8.2,40.24,1985
2,3,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,8.0,7.5,40.00,2012
3,4,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,8.2,9.1,37.32,2008
4,5,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,8.6,4.7,36.60,2017


In [3]:
df_games.to_sql(
            'game_sales_data',            
             conn,                
             if_exists='replace'
            )

19600

In [4]:
cursor = conn.cursor()

The **Video Game Sales Dataset** provides comprehensive information on video game sales, spanning multiple platforms and publishers. It includes data on over **16,500 video games** released between **1977 and 2020**. The dataset covers key attributes such as: 

- **Game Information**:  
  - `Name`: Title of the game (13.6k unique values, most common: *Minecraft*).  
  - `Platform`: Gaming platform (e.g., DS, PS2, Xbox, etc., 37 unique platforms).  
  - `Publisher` & `Developer`: Information on the companies behind the game (e.g., Activision, Ubisoft).  

- **Sales Data**:  
  - `Total_Shipped`: Number of units sold (ranging from **0.01M to 82.9M** copies).  
  - `Rank`: Sales ranking based on total units sold.  

- **Ratings & Reviews**:  
  - `Critic_Score`: Scores given by professional critics (ranging from **0.8 to 10**, average **7.03**).  
  - `User_Score`: Ratings given by players (**1 to 10**, average **6.99**).  


# 📊 Database Overview

This script analyzes the `game_sales_data` table and provides a statistical summary of each column, including:

- **Column Name**: The name of the column.
- **Missing Values**: The number of missing (NULL) values in the column.
- **Distinct Values**: The number of unique values in the column.
- **Min Value**: The smallest value in the column (numerically or alphabetically).
- **Max Value**: The largest value in the column (numerically or alphabetically).

This overview helps understand data quality, detect missing values, and analyze the distribution of values in the dataset.



In [5]:
data_overview = []

for col in df_columns:
    query = f"""
        SELECT 
            COUNT(*) - COUNT({col}) AS missing_values,
            COUNT(DISTINCT {col}) AS distinct_values,
            MIN({col}) AS min_value,
            MAX({col}) AS max_value
        FROM game_sales_data;
    """
    data_sql_col = pd.read_sql(query, conn)

    data_length = pd.read_sql("""SELECT COUNT(*) AS count FROM game_sales_data""", conn)
    
    data_overview.append([
        col, 
        round(data_sql_col['missing_values'][0] / data_length['count'][0] *100, 2), 
        data_sql_col['distinct_values'][0], 
        data_sql_col['min_value'][0] , 
        data_sql_col['max_value'][0]
    ])

df_overview = pd.DataFrame(data_overview, columns=["Column", "Missing Values", "Distinct Values", "Min Value", "Max Value"])
display(df_overview)


,Column,Missing Values,Distinct Values,Min Value,Max Value
0,Rank,0.00,19598,1,19598
1,Name,0.00,13612,'98 Koshien,uDraw Studio: Instant Artist
2,Platform,0.00,37,2600,XOne
3,Publisher,0.00,1012,10TACLE Studios,team BitClub
4,Developer,0.01,3054,07th Expansion,zSlide
5,Critic_Score,49.14,90,0.8,10.0
6,User_Score,88.66,88,1.0,10.0
7,Total_Shipped,0.00,664,0.01,82.9
8,Year,0.00,44,1977,2020


## 🎮 Exploring Game Sales: *Grand Theft Auto V*

The following query retrieves all available records for **Grand Theft Auto V**, showing its sales performance across different platforms:  


In [6]:
cursor = conn.cursor()
data_sql_names = pd.read_sql("""
    SELECT * 
    FROM game_sales_data 
    WHERE Name = 'Grand Theft Auto V';
""", conn)

display(data_sql_names)

,index,Rank,Name,Platform,Publisher,Developer,Critic_Score,User_Score,Total_Shipped,Year
0,22,23,Grand Theft Auto V,PS3,Rockstar Games,Rockstar North,9.4,8.3,20.32,2013
1,23,24,Grand Theft Auto V,PS4,Rockstar Games,Rockstar North,9.7,8.4,19.39,2014
2,40,41,Grand Theft Auto V,X360,Rockstar Games,Rockstar North,10.0,NaN,15.86,2013
3,70,71,Grand Theft Auto V,PC,Rockstar Games,Rockstar North,9.6,7.7,12.60,2015
4,122,123,Grand Theft Auto V,XOne,Rockstar Games,Rockstar North,9.0,9.0,8.72,2014


- **Multi-Platform Success**: *Grand Theft Auto V* was released on multiple platforms, including **PS3, PS4, Xbox 360, Xbox One, and PC**.  
- **Strong Sales**: The game **sold over 20 million copies on PS3 alone**, with notable sales across all other platforms.  
- **High Critic Scores**: The game received **consistently high critic scores (9.0+ across all platforms)**, peaking at **10.0 on Xbox 360**.  
- **Varying User Scores**: While most versions have **strong user scores**, the **PC version has a slightly lower rating (7.7)**, possibly due to **technical issues at launch**.  
- **Missing Data**: The **User Score for the Xbox 360 version is missing**, highlighting potential gaps in the dataset.  

# The 10 Best-Selling Video Games  

The following SQL query retrieves the **10 best-selling video games** based on total units shipped. It groups the dataset by game name and calculates key metrics:  

- **First Release Year (`first_year`)**: The earliest release year of the game.  
- **Number of Platforms (`nbr_platforms`)**: The number of different platforms the game was released on.  
- **Total Units Shipped (`sum_total_shipped`)**: The total sales in millions of copies.  
- **Minimum & Maximum Critic Scores (`min_critic_score`, `max_critic_score`)**: The lowest and highest critic scores recorded for the game.  


In [7]:
data_sql_vg = pd.read_sql("""
    SELECT Name,
            MIN(Year) AS first_year,
            COUNT(*) AS nbr_platforms,
            SUM(Total_Shipped) AS sum_total_shipped,
            MIN(Critic_Score) AS min_critic_score,
            MAX(Critic_Score) AS max_critic_score
    FROM game_sales_data 
    GROUP BY Name
    ORDER BY sum_total_shipped DESC
    LIMIT 10;
""", conn)

display(data_sql_vg)

,Name,first_year,nbr_platforms,sum_total_shipped,min_critic_score,max_critic_score
0,Wii Sports,2006,1,82.90,7.7,7.7
1,Grand Theft Auto V,2013,5,76.89,9.0,10.0
2,Minecraft,2010,9,70.16,5.5,10.0
3,Super Mario Bros.,1985,1,40.24,10.0,10.0
4,Counter-Strike: Global Offensive,2012,1,40.00,8.0,8.0
5,Mario Kart Wii,2008,1,37.32,8.2,8.2
6,PLAYERUNKNOWN'S BATTLEGROUNDS,2017,1,36.60,8.6,8.6
7,Tetris,1989,2,35.84,9.4,9.4
8,Wii Sports Resort,2009,1,33.13,8.0,8.0
9,Pokemon Red / Green / Blue Version,1998,1,31.38,9.4,9.4


- **Wii Sports is the Best-Selling Game**: With **82.9M units sold**, it surpasses *GTA V* and *Minecraft*.  
- **Multi-Platform Games Sell More**: *Grand Theft Auto V* and *Minecraft* achieved high sales due to their availability on **multiple platforms** (5 and 9, respectively).  
- **High-Critic Score Classics**: *Super Mario Bros.* and *Tetris* maintain a **perfect 10/10 critic score**, reflecting their **enduring legacy**.  
- **Newer Hits vs. Classics**: *PLAYERUNKNOWN'S BATTLEGROUNDS (2017)* stands out as a modern best-seller, while *Super Mario Bros. (1985)* still holds a top position after **decades**.  
- **Variability in Critic Scores**: *Minecraft* has a **low critic score of 5.5**, possibly due to earlier versions, but reaches **10.0** on certain platforms.  

# Years that Received Critical Acclaim in Video Games

In this section, we analyzed the evolution of video game releases and review scores over time. Using SQL, we extracted yearly aggregated data from a video game sales database, calculating key metrics such as:  

- **Number of games released per year**  
- **Average critic and user scores**  
- **Number of unique platforms and publishers**  

The data was then visualized using **Plotly**, with a dual-axis chart:  
- **A bar chart** representing the number of games released each year.  
- **Line charts** tracking the evolution of average critic and user scores.  

This analysis provides insights into industry trends, user satisfaction, and how game releases have evolved over the years.  


In [ ]:
data_sql_vg = pd.read_sql("""
    SELECT Year,
           COUNT(*) AS nbr_games_released,
            COUNT(DISTINCT Platform) AS nbr_platforms,
            COUNT(DISTINCT Publisher) AS nbr_publisher,
            ROUND(AVG(Critic_Score), 2) AS avg_critic_score,
            ROUND(AVG(User_Score), 2) AS avg_user_score        
    FROM game_sales_data
    GROUP BY Year
    ;
""", conn)

display(data_sql_vg.loc[(data_sql_vg.Year > 1995) & (data_sql_vg.Year < 2015)])

,Year,nbr_games_released,nbr_platforms,nbr_publisher,avg_critic_score,avg_user_score
19,1996,291,8,82,6.76,8.50
20,1997,264,6,66,6.79,9.25
21,1998,342,8,86,7.15,8.80
22,1999,359,9,75,7.19,9.10
23,2000,412,8,73,7.06,8.45
24,2001,508,11,84,7.21,7.77
25,2002,821,11,92,7.02,8.48
26,2003,813,6,80,7.27,8.53
27,2004,785,8,83,7.14,7.91
28,2005,964,9,91,7.11,7.62


In [30]:
fig = go.Figure()

# Histogramme pour le nombre de jeux sortis sur un axe séparé
fig.add_trace(go.Bar(
    x=data_sql_vg["Year"],
    y=data_sql_vg["nbr_games_released"],
    name="Number of Games Released",
    marker=dict(color="rgba(121, 85, 72, 0.3)"), 
    opacity=0.5,
    yaxis='y2'
))

# Courbe pour le score des critiques (couleur plus foncée)
fig.add_trace(go.Scatter(
    x=data_sql_vg["Year"],
    y=data_sql_vg["avg_critic_score"],
    mode="lines+markers",
    name="Average Critic Score",
    line=dict(color="#D8B6A4", width=3),  
    marker=dict(size=8, color="#D8B6A4")  
))

# Courbe pour le score des utilisateurs (couleur plus foncée)
fig.add_trace(go.Scatter(
    x=data_sql_vg["Year"],
    y=data_sql_vg["avg_user_score"],
    mode="lines+markers",
    name="Average User Score",
    line=dict(color="#795548", width=3),  
    marker=dict(size=8, color="#795548") 
))

# Mise en forme du graphique avec un axe séparé
fig.update_layout(
    title="🎮 Evolution of Critic & User Scores Over Time",
    xaxis_title="Year",
    yaxis=dict(
        title="Score (0-10)",
        showgrid=True
    ),
    yaxis2=dict(
        title="Number of Games Released",
        overlaying='y',
        side='right',
        showgrid=False
    ),
    template="plotly_white",
    font=dict(family="Arial, sans-serif", size=14, color="#4A4A4A"),
    plot_bgcolor="#EFE7E1",  
    paper_bgcolor="#EFE7E1",
    hovermode="x",
    legend=dict(title="Score Type", orientation="h", x=0.5, xanchor="center", y=-0.2),
    margin=dict(l=40, r=40, t=60, b=40)
)

# Affichage du graphique
fig.show()



1️⃣ **Trends in Game Releases**  
- The number of games released **steadily increased from the late 1990s**, reaching a **peak around 2008-2009** with over **1,500 games released** in a single year.  
- After 2010, the number of games released **declined significantly**, stabilizing at lower levels in the 2010s.  
- Possible reasons for this decline:
  - A shift towards **higher-quality, longer development cycle games**.
  - The rise of **live-service games** (fewer new releases, but ongoing content updates).  
  - Industry consolidation, with **fewer publishers dominating the market**.  

2️⃣ **Evolution of Critic and User Scores**  
- **Before 2005**, the scores aren't very representative given the low number of games released.   
- **2006-2010**:  
  - **Critic scores declined**, possibly due to an oversaturation of games with **lower quality releases**.  
  - **User scores remained relatively high (~8.0+).**  
- **2011-2015**:  
  - A sharp **drop in user scores** after 2011 (below 7.0).  
  - Critic scores remained stable, suggesting a **growing disconnect between critics and players**.  
  - This period coincides with **controversial industry practices** (e.g., microtransactions, pay-to-win mechanics, day-one DLCs).  
- **2016-Present**:  
  - A **slight recovery in user scores**, but critic scores remained consistent.  
  - The number of game releases remained **much lower than in the early 2000s**, suggesting a shift towards **quality over quantity**.  

3️⃣ **Key Observations**  
- **The golden era of user satisfaction (1996-2005)** shows the highest average user scores.  
- **Post-2010 decline** in both releases and user scores suggests a shift in the gaming industry.  
- **Critics vs. users gap**: After 2010, critics’ scores remained stable while user scores dropped—possibly reflecting growing dissatisfaction among players.  
- **2020s recovery?** The recent uptick in user scores could indicate a positive shift in the industry.  

# Industry Trends: Most Popular Platforms, Games, and Publishers by Year  

This SQL query examines **the evolution of the video game industry**, highlighting the platforms with the most releases, the best-selling games, and the leading publishers each year. It also provides an overview of **total game releases and sales**, offering insights into market dynamics over time.  

To efficiently compute these rankings, the query uses **common table expressions (CTEs)** before consolidating the results into a final dataset. The output includes:  

- **Year** – The release year, establishing a timeline for analysis.  
- **Total Games Released (`total_games`)** – A measure of industry activity, indicating the volume of new titles launched annually.  
- **Total Sales (`total_sales`)** – The total number of game units shipped that year, reflecting overall market performance.  
- **Most Popular Platform (`most_popular_platform`)** – The system that hosted the highest number of releases.  
- **Top Publishers (`Publisher1`, `Publisher2`)** – The companies generating the most sales in a given year.  
- **Best-Selling Game (`most_popular_game`)** – The title with the highest total units shipped.  



In [ ]:
cursor = conn.cursor()
data_sql_names = pd.read_sql("""
    WITH YearCountPlatform AS(
                        SELECT
                             Year, Platform AS most_popular_platform,
                             RANK() OVER (PARTITION BY Year ORDER BY COUNT(*) DESC) AS rank_platform
                        FROM game_sales_data
                        GROUP BY Year, Platform 
    ),
    YearCountGames AS(
                        SELECT
                             Year, Name AS most_popular_game,
                             RANK() OVER (PARTITION BY Year ORDER BY SUM(Total_Shipped) DESC) AS rank_games
                        FROM game_sales_data
                        GROUP BY Year, Name 
    ),
    PublisherRanking AS (
                        SELECT Publisher, Year,  
                            RANK() OVER(PARTITION BY Year ORDER BY SUM(Total_Shipped) DESC) AS rank
                        FROM game_sales_data
                        GROUP BY Publisher, Year
    ),
    PublisherRanking2 AS (
                        SELECT Year,
                             MAX(CASE WHEN rank = 1 THEN Publisher END) AS Publisher1,
                             MAX(CASE WHEN rank = 2 THEN Publisher END) AS Publisher2
                        FROM PublisherRanking GROUP BY Year
    ),
    StatYear AS (
                        SELECT Year, COUNT(*) AS total_games, 
                             SUM(Total_Shipped) AS total_sales 
                        FROM game_sales_data
                        GROUP BY Year
    )
    SELECT s.Year, 
            s.total_games, 
            s.total_sales,
            p.most_popular_platform, 
            pr.Publisher1, 
            pr.Publisher2, 
            g.most_popular_game
    FROM StatYear s 
    JOIN YearCountPlatform p ON p.Year = s.Year
    JOIN YearCountGames g ON g.Year = s.Year   
    JOIN PublisherRanking2 pr ON pr.Year = s.Year
    WHERE (g.rank_games = 1) AND (p.rank_platform = 1)
    ORDER BY s.Year;
""", conn)

display(data_sql_names)

,Year,total_games,total_sales,most_popular_platform,Publisher1,Publisher2,most_popular_game
0,1977,3,2.50,2600,Atari,None,Combat
1,1978,9,6.19,2600,Atari,Sears,Space Invaders
2,1979,1,0.31,2600,Atari,None,Slot Machine
3,1980,6,2.58,2600,Activision,Unknown,Boxing
4,1981,7,10.49,2600,Atari,Activision,Asteroids
5,1982,47,43.39,2600,Atari,Activision,Pac-Man
6,1983,38,24.82,2600,Atari,Activision,Mahjong
7,1984,12,7.82,2600,Atari,Nintendo,F1 Race
8,1985,15,93.72,NES,Nintendo,Activision,Super Mario Bros.
9,1986,18,21.40,NES,Nintendo,Capcom,Super Mario Bros. 2 (FDS)


- **Nintendo dominated the industry for decades**, leading both in platform usage and best-selling games.  
- **Sony & Microsoft platforms took over in later years**, especially with the **PS2, PS3, and PS4 eras**.  
- **The most successful publishers varied** over time, with **EA, Ubisoft, Activision, and Nintendo** consistently ranking among the top.  
- **Some games had multi-year dominance**, such as *Grand Theft Auto V* and *Minecraft*.  
